In [1]:
import comet_ml
import torch as th
import torch.nn.functional as F
from torch.utils.data import SubsetRandomSampler
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger, CometLogger

from models import MULTModel
from datasets import load_impressionv2_dataset_all, load_impressionv2_dataset_split

In [2]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('-f', default='', type=str)

# Fixed
parser.add_argument('--model', type=str, default='MulT',
                    help='name of the model to use (Transformer, etc.)')

# Tasks
parser.add_argument('--vonly', action='store_true',
                    help='use the crossmodal fusion into v (default: False)')
parser.add_argument('--aonly', action='store_true',
                    help='use the crossmodal fusion into a (default: False)')
parser.add_argument('--lonly', action='store_true',
                    help='use the crossmodal fusion into l (default: False)')
# parser.add_argument('--aligned', action='store_true',
#                     help='consider aligned experiment or not (default: False)')
# parser.add_argument('--dataset', type=str, default='mosei_senti',
#                     help='dataset to use (default: mosei_senti)')
# parser.add_argument('--data_path', type=str, default='data',
#                     help='path for storing the dataset')

# Dropouts
parser.add_argument('--attn_dropout', type=float, default=0.1,
                    help='attention dropout')
parser.add_argument('--attn_dropout_a', type=float, default=0.0,
                    help='attention dropout (for audio)')
parser.add_argument('--attn_dropout_v', type=float, default=0.0,
                    help='attention dropout (for visual)')
parser.add_argument('--relu_dropout', type=float, default=0.1,
                    help='relu dropout')
parser.add_argument('--embed_dropout', type=float, default=0.25,
                    help='embedding dropout')
parser.add_argument('--res_dropout', type=float, default=0.1,
                    help='residual block dropout')
parser.add_argument('--out_dropout', type=float, default=0.0,
                    help='output layer dropout')

# Architecture
parser.add_argument('--layers', type=int, default=5,
                    help='number of layers in the network (default: 5)')
parser.add_argument('--num_heads', type=int, default=5,
                    help='number of heads for the transformer network (default: 5)')
parser.add_argument('--attn_mask', action='store_false',
                    help='use attention mask for Transformer (default: true)')

# Tuning
parser.add_argument('--batch_size', type=int, default=2, metavar='N',
                    help='batch size (default: 24)')
# parser.add_argument('--clip', type=float, default=0.8,
#                     help='gradient clip value (default: 0.8)')
parser.add_argument('--lr', type=float, default=1e-3,
                    help='initial learning rate (default: 1e-3)')
# parser.add_argument('--optim', type=str, default='Adam',
#                     help='optimizer to use (default: Adam)')
parser.add_argument('--num_epochs', type=int, default=40,
                    help='number of epochs (default: 40)')
# parser.add_argument('--when', type=int, default=20,
#                     help='when to decay learning rate (default: 20)')
# parser.add_argument('--batch_chunk', type=int, default=1,
#                     help='number of chunks per batch (default: 1)')

# Logistics
# parser.add_argument('--log_interval', type=int, default=30,
#                     help='frequency of result logging (default: 30)')
# parser.add_argument('--seed', type=int, default=1111,
#                     help='random seed')
# parser.add_argument('--no_cuda', action='store_true',
#                     help='do not use cuda')
parser.add_argument('--name', type=str, default='mult',
                    help='name of the trial (default: "mult")')
args = parser.parse_args()

valid_partial_mode = args.lonly + args.vonly + args.aonly

if valid_partial_mode == 0:
    args.lonly = args.vonly = args.aonly = True
elif valid_partial_mode != 1:
    raise ValueError("You can only choose one of {l/v/a}only.")

del args.f
hyp_params = args

In [3]:
class MULTModelWarped(pl.LightningModule):
    def __init__(self, hyp_params):
        super().__init__()
        self.model = MULTModel(hyp_params)
        self.save_hyperparameters(hyp_params)
        
    def forward(self, *args):
        if len(args) == 3:
            text, audio, face = args
        else:
            text, audio, face = args[0]
        return self.model(text, audio, face)[0]
    
    def configure_optimizers(self):
        optimizer = th.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        audio, face, text, y = batch
        y_hat = self(text, audio, face)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        audio, face, text, y = batch
        y_hat = self(text, audio, face)
        loss = F.mse_loss(y_hat, y)
        self.log('valid_loss', loss)
        return loss
    
    def test_step(self, batch, batch_idx):
        audio, face, text, y = batch
        y_hat = self(text, audio, face)
        loss = F.mse_loss(y_hat, y)
        self.log('test_loss', loss)
        return loss

In [4]:
train_ds, valid_ds, test_ds = load_impressionv2_dataset_all()

In [5]:
hyp_params.batch_size = 16

In [6]:
train_dl = th.utils.data.DataLoader(train_ds, batch_size=hyp_params.batch_size, pin_memory=True)
valid_dl = th.utils.data.DataLoader(valid_ds, batch_size=hyp_params.batch_size, pin_memory=True)
test_dl = th.utils.data.DataLoader(test_ds, batch_size=hyp_params.batch_size, pin_memory=True)

In [7]:
audio, face, text, label = next(iter(train_dl))
hyp_params.orig_d_l = text.shape[2]
hyp_params.orig_d_a = audio.shape[2]
hyp_params.orig_d_v = face.shape[2]
hyp_params.l_len = text.shape[1]
hyp_params.a_len = audio.shape[1]
hyp_params.v_len = face.shape[1]
# hyp_params.use_cuda = True
# hyp_params.dataset = dataset
# hyp_params.when = args.when
# hyp_params.batch_chunk = args.batch_chunk
# hyp_params.n_train, hyp_params.n_valid, hyp_params.n_test = len(train_data), len(valid_data), len(test_data)
# hyp_params.model = str.upper(args.model.strip())
hyp_params.output_dim = label.shape[1] #output_dim_dict.get(dataset, 1)
# hyp_params.criterion = th.nn.L1Loss #criterion_dict.get(dataset, 'L1Loss')

In [8]:
model = MULTModelWarped(hyp_params)
csv_logger = CSVLogger("logs", name="my_exp_name")
comet_logger = CometLogger(api_key="cgss7piePhyFPXRw1J2uUEjkQ", workspace="transformer", project_name="find_batchsize")
trainer = pl.Trainer(gpus=1, max_epochs=hyp_params.num_epochs, log_every_n_steps=1, logger=[csv_logger, comet_logger], fast_dev_run=4)
trainer.fit(model, train_dl, valid_dl)

CometLogger will be initialized in online mode
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 4 batch(es).

  | Name  | Type      | Params
------------------------------------
0 | model | MULTModel | 1.1 M 
------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.405     Total estimated model params size (MB)
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not

/workspace/transformer/modules/position_embedding.py:23: UserWarning: The number of elements in the out tensor of shape [60] is 60 which does not match the computed number of elements 1526. Note that this may occur as a result of rounding error. The out tensor will be resized to a tensor of shape (1526,). (Triggered internally at  /pytorch/aten/src/ATen/native/cuda/RangeFactories.cu:266.)
  torch.arange(padding_idx + 1, max_pos, out=getattr(make_positions, buf_name))


RuntimeError: CUDA out of memory. Tried to allocate 712.00 MiB (GPU 0; 10.76 GiB total capacity; 9.05 GiB already allocated; 423.00 MiB free; 9.33 GiB reserved in total by PyTorch)

In [ ]:
trainer.test(test_dataloaders=test_dl)